In [1]:
#importamos las librerias necesarias para el proceso

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 

In [2]:

#cargamos los dataset necesarios para el trabajo. 
df_demographics = pd.read_csv('data/raw/df_final_demo.txt', sep=',')
df_experiment = pd.read_csv('data/raw/df_final_experiment_clients.txt', sep=',')

#Carga de las dos partes 
df_webdata1 = pd.read_csv('data/raw/df_final_web_data_pt_1.txt', sep=',')
df_webdata2 = pd.read_csv('data/raw/df_final_web_data_pt_2.txt', sep=',')

### Cargamos dataset demograficos. 

In [3]:
#Vemos una muestra de los datos.
df_demographics.head(5)

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,836976,6.0,73.0,60.5,U,2.0,45105.30,6.0,9.0
1,2304905,7.0,94.0,58.0,U,2.0,110860.30,6.0,9.0
2,1439522,5.0,64.0,32.0,U,2.0,52467.79,6.0,9.0
3,1562045,16.0,198.0,49.0,M,2.0,67454.65,3.0,6.0
4,5126305,12.0,145.0,33.0,F,2.0,103671.75,0.0,3.0


In [4]:
#hacemos un chuequeo inicial de los datos. 
def data_exploration(df):

    # check number of rows and columns
    shape = df.shape
    print("Number of rows:", shape[0])
    print("Number of columns:", shape[1])

    # check duplicates
    check_duplicates = df.duplicated().sum()
    print("Number of duplicates:", check_duplicates)

    # Create a summary DataFrame
    summary_df = pd.DataFrame({
                        'Column': df.columns,
                        'Data Type': df.dtypes,
                        'Non-Null Count': df.notnull().sum(),
                        'Missing Values': df.isnull().sum(),
                        'Unique Values': df.nunique()
                })

    # Reset index to make 'Column' a regular column
    summary_df.reset_index(drop=True, inplace=True)

    # Display the summary DataFrame
    summary_df
    
    # check numerical columns
    numerical_columns = df.select_dtypes("number").columns
    print("\nNumerical Columns:", numerical_columns)

    # check categorical columns
    categorical_columns = df.select_dtypes("object").columns
    print("\nCategorical Columns:", categorical_columns)
    # Print summary statistics for numerical columns
    print("\nSummary Statistics:\n")
    print(df.describe())

    return summary_df

In [5]:
data_exploration(df_demographics)

Number of rows: 70609
Number of columns: 9
Number of duplicates: 0

Numerical Columns: Index(['client_id', 'clnt_tenure_yr', 'clnt_tenure_mnth', 'clnt_age',
       'num_accts', 'bal', 'calls_6_mnth', 'logons_6_mnth'],
      dtype='object')

Categorical Columns: Index(['gendr'], dtype='object')

Summary Statistics:

          client_id  clnt_tenure_yr  clnt_tenure_mnth      clnt_age  \
count  7.060900e+04    70595.000000      70595.000000  70594.000000   
mean   5.004992e+06       12.052950        150.659367     46.442240   
std    2.877278e+06        6.871819         82.089854     15.591273   
min    1.690000e+02        2.000000         33.000000     13.500000   
25%    2.519329e+06        6.000000         82.000000     32.500000   
50%    5.016978e+06       11.000000        136.000000     47.000000   
75%    7.483085e+06       16.000000        192.000000     59.000000   
max    9.999839e+06       62.000000        749.000000     96.000000   

          num_accts           bal  calls_6_

,Column,Data Type,Non-Null Count,Missing Values,Unique Values
0,client_id,int64,70609,0,70609
1,clnt_tenure_yr,float64,70595,14,54
2,clnt_tenure_mnth,float64,70595,14,482
3,clnt_age,float64,70594,15,165
4,gendr,object,70595,14,4
5,num_accts,float64,70595,14,8
6,bal,float64,70595,14,70328
7,calls_6_mnth,float64,70595,14,8
8,logons_6_mnth,float64,70595,14,9


- Analizamos por separado las columnas categoricas y numericas. 

In [6]:
def analyze_numerical(df: pd.DataFrame) -> pd.DataFrame:
    # Select numerical columns
    numerical_cols = df.select_dtypes(include=['number']).columns

    # Perform descriptive analysis on numerical columns
    numerical_desc = df[numerical_cols].describe()

    # Display the resulting DataFrame
    print("\nNumerical Columns Analysis:")

    return numerical_desc

analyze_numerical(df_demographics)


Numerical Columns Analysis:


,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,num_accts,bal,calls_6_mnth,logons_6_mnth
count,7.060900e+04,70595.000000,70595.000000,70594.000000,70595.000000,7.059500e+04,70595.000000,70595.000000
mean,5.004992e+06,12.052950,150.659367,46.442240,2.255528,1.474452e+05,3.382478,5.566740
std,2.877278e+06,6.871819,82.089854,15.591273,0.534997,3.015087e+05,2.236580,2.353286
min,1.690000e+02,2.000000,33.000000,13.500000,1.000000,1.378942e+04,0.000000,1.000000
25%,2.519329e+06,6.000000,82.000000,32.500000,2.000000,3.734683e+04,1.000000,4.000000
50%,5.016978e+06,11.000000,136.000000,47.000000,2.000000,6.333290e+04,3.000000,5.000000
75%,7.483085e+06,16.000000,192.000000,59.000000,2.000000,1.375449e+05,6.000000,7.000000
max,9.999839e+06,62.000000,749.000000,96.000000,8.000000,1.632004e+07,7.000000,9.000000


In [7]:
def analyze_categorical(df: pd.DataFrame) -> pd.DataFrame:
    # Select categorical columns
    categorical_cols = df.select_dtypes(include=['object', 'category'])

    # Perform descriptive analysis on categorical columns
    categorical_desc = categorical_cols.describe()

    return categorical_desc

analyze_categorical(df_demographics)

,gendr
count,70595
unique,4
top,U
freq,24122


In [8]:
def calculate_null_percentage(df):
    # Calcular el porcentaje de nulos
    null_percentage = df.isnull().mean().sort_values(ascending=False) * 100
    
    # Formatear los valores con dos decimales y el símbolo %
    null_percentage_formatted = null_percentage.apply(lambda x: f"{x:.2f}%")
    
    print("Porcentaje de valores nulos por columna:")
    return null_percentage_formatted

calculate_null_percentage(df_demographics)

Porcentaje de valores nulos por columna:


clnt_age            0.02%
num_accts           0.02%
clnt_tenure_yr      0.02%
clnt_tenure_mnth    0.02%
gendr               0.02%
calls_6_mnth        0.02%
bal                 0.02%
logons_6_mnth       0.02%
client_id           0.00%
dtype: object

In [9]:
#limpiamos los datos demograficos y los guardamos en un nuevo csv para trabajar con ellos.

def limpiar_demograficos(df):
    # Renombra columnas para mayor claridad
    df = df.rename(columns={
        'clnt_tenure_yr': 'client_tenure_years',
        'clnt_tenure_mnth': 'client_tenure_months',
        'clnt_age': 'age',
        'gendr': 'gender',
        'num_accts': 'number_of_accounts',
        'bal': 'balance',
    })

    # Elimina filas con valores nulos (menos del 0.02% del total)
    df = df.dropna()

    # Normaliza género: reemplaza 'X' por 'U' (Unknown)
    df["gender"] = df["gender"].replace({"X": "U"})

    # Convierte la edad a entero truncado (elimina .5)
    df["age"] = df["age"].astype(int)

    return df


df_demo_clean = limpiar_demograficos(df_demographics)
df_demo_clean.to_csv('data/cleaned/df_final_demo_clean.csv', index=False)

### Cargamos el dataset del paso de los clientes en el nuevo proceso. 

In [10]:
# Hacemos el merge de los datos de webdata y los limpiamos. 

def merge_dataframes(df1, df2, key='client_id'):
    
    merged_df = pd.merge(df1, df2, how='inner', on=key)

    # Eliminar columnas duplicadas que terminan en '_y'
    cols_to_drop = [col for col in merged_df.columns if col.endswith('_y')]
    merged_df.drop(columns=cols_to_drop, inplace=True)

    # Renombrar columnas que terminan en '_x' eliminando el sufijo
    merged_df.rename(columns={
        col: col.replace('_x', '') for col in merged_df.columns if col.endswith('_x')
    }, inplace=True)
    
    # Eliminar duplicados y ordenar por client_id y date_time
    merged_df = merged_df.drop_duplicates().sort_values(['client_id', 'date_time'])

    return merged_df

# Llamamos a la funcion para hacer el merge de los datos 
df_final_webdata = merge_dataframes(df_webdata1, df_webdata2)

# se guarda en la carpeta files para usarlo en el futuro 
df_final_webdata.to_csv('data/cleaned/df_final_web_data.csv', index=False)

#Cargamos el dataframe final para comprobar que se ha guardado correctamente. 
df_final_webdata = pd.read_csv('data/cleaned/df_final_web_data.csv',sep=',')
df_final_webdata.head(10)

,client_id,visitor_id,visit_id,process_step,date_time
0,1643,910842861_51062379179,379946188_1773022140_107963,start,2017-04-15 15:06:00
1,1643,910842861_51062379179,633860590_96880450633_976109,start,2017-04-24 06:36:07
2,1643,910842861_51062379179,633860590_96880450633_976109,step_1,2017-04-24 06:36:32
3,1643,910842861_51062379179,633860590_96880450633_976109,start,2017-04-24 06:37:17
4,1643,910842861_51062379179,633860590_96880450633_976109,start,2017-04-24 06:38:20
5,1677,361240963_15830034505,494607202_74539107264_966618,start,2017-03-31 10:59:27
6,1677,361240963_15830034505,494607202_74539107264_966618,step_1,2017-03-31 10:59:41
7,1677,361240963_15830034505,494607202_74539107264_966618,step_2,2017-03-31 11:00:40
8,1677,361240963_15830034505,494607202_74539107264_966618,start,2017-03-31 11:01:50
9,1680,262688118_24224945474,807102410_81976926205_767113,start,2017-03-15 09:57:21


In [11]:
# Comprobamos el dataset final de webdata llamando a la funcion de exploracion de datos. 
data_exploration(df_final_webdata)

Number of rows: 28199
Number of columns: 5
Number of duplicates: 0

Numerical Columns: Index(['client_id'], dtype='object')

Categorical Columns: Index(['visitor_id', 'visit_id', 'process_step', 'date_time'], dtype='object')

Summary Statistics:

          client_id
count  2.819900e+04
mean   5.034490e+06
std    2.849419e+06
min    1.643000e+03
25%    2.601296e+06
50%    5.082650e+06
75%    7.447604e+06
max    9.999729e+06


,Column,Data Type,Non-Null Count,Missing Values,Unique Values
0,client_id,int64,28199,0,5664
1,visitor_id,object,28199,0,6450
2,visit_id,object,28199,0,8290
3,process_step,object,28199,0,5
4,date_time,object,28199,0,27787


### Cargamos el dataset de los datos del experimento. 
- Contienen el grupo control y el grupo test de los usuarios. 
    

In [12]:
df_experiment.head(10)

,client_id,Variation
0,9988021,Test
1,8320017,Test
2,4033851,Control
3,1982004,Test
4,9294070,Control
5,9466803,Test
6,3170143,Test
7,6651403,Control
8,5887227,Test
9,2105948,Control


In [13]:
data_exploration(df_experiment)

Number of rows: 70609
Number of columns: 2
Number of duplicates: 0

Numerical Columns: Index(['client_id'], dtype='object')

Categorical Columns: Index(['Variation'], dtype='object')

Summary Statistics:

          client_id
count  7.060900e+04
mean   5.004992e+06
std    2.877278e+06
min    1.690000e+02
25%    2.519329e+06
50%    5.016978e+06
75%    7.483085e+06
max    9.999839e+06


,Column,Data Type,Non-Null Count,Missing Values,Unique Values
0,client_id,int64,70609,0,70609
1,Variation,object,50500,20109,2


In [14]:
calculate_null_percentage(df_experiment)

Porcentaje de valores nulos por columna:


Variation    28.48%
client_id     0.00%
dtype: object

In [15]:
#Borramos los datos nulos de la columna 'experiment' y los guardamos en un nuevo csv.
df_clients = df_experiment.dropna(subset=["Variation"])
df_clients.to_csv("data/cleaned/df_final_experiment_clients_cleaned.csv", index=False)